In [1]:
import json
import pandas as pd
import numpy as np
import pickle
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter
from time import time

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)
pd.set_option('max_colwidth', 200)

In [3]:
pickle_in = open(r"C:\Users\surya\Desktop\Jupyter Python Programs\Amazon Review NLP\Review_features_data_1.pickle", 'rb')
df = pickle.load(pickle_in)
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_rank
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,"Looks even better in person. Be careful to not drop your phone so often because the rhinestones will fall off (duh). More of a decorative case than it is protective, but I will say that it fits pe...",Can't stop won't stop looking at it,1407110400,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back Cover Case Skin For Apple iPhone 5 5G', 'Ultra light weight case', 'Elegantly designed with 3D cutout effect', 'Super rich rubber coating']","['Fashionable with unique 3D butterfly design for the iPhone 5 from others', 'Precise cut design with access to all ports and features of the iPhone 5', 'Dirt-resistant Butterfly Case for iPhone 5...","['>#228,544 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']",Cell Phones & Accessories,"['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL._SX38_SY50_CR,0,0,38,50_.jpg']",['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL.jpg'],4,22,21,228544,"Cases, Holsters & Clips",NaN,NaN,114920
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of cash but want a great deal...this is the shop to buy from!,1,1392163200,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back Cover Case Skin For Apple iPhone 5 5G', 'Ultra light weight case', 'Elegantly designed with 3D cutout effect', 'Super rich rubber coating']","['Fashionable with unique 3D butterfly design for the iPhone 5 from others', 'Precise cut design with access to all ports and features of the iPhone 5', 'Dirt-resistant Butterfly Case for iPhone 5...","['>#228,544 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']",Cell Phones & Accessories,"['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL._SX38_SY50_CR,0,0,38,50_.jpg']",['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL.jpg'],1,10,9,228544,"Cases, Holsters & Clips",NaN,NaN,114920
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I'm actually missing 2 studs but nothing too noticeable the studding is almost a bit sloppy around the bow, but once again not too noticeable. I haven'...",Its okay,1391817600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back Cover Case Skin For Apple iPhone 5 5G', 'Ultra light weight case', 'Elegantly designed with 3D cutout effect', 'Super rich rubber coating']","['Fashionable with unique 3D butterfly design for the iPhone 5 from others', 'Precise cut design with access to all ports and features of the iPhone 5', 'Dirt-resistant Butterfly Case for iPhone 5...","['>#228,544 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']",Cell Phones & Accessories,"['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL._SX38_SY50_CR,0,0,38,50_.jpg']",['https://images-na.ssl-images-amazon.com/images/I/41Wol0yPcxL.jpg'],3,22,21,228544,"Cases, Holsters & Clips",NaN,NaN,114920
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY WERE OKAY WITH IT. JUST NOT WHAT I EXPECTED.,CASE,1391472000,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back Cover Case Skin For Apple iPhone 5 5G', 'Ultra light weight case', 'Elegantly designed with 3D cutout effect', 'Super rich rubber coating']","['Fashionable with unique 3D butterfly design for the iPhone 5 from others', 'Precise cut design with access to all ports and features of the iPhone 5', 'Dirt-resistant Butterfly Case f

In [4]:
df['sub_cat_2'].unique()

array([nan, ' Basic Cases', ' Chargers & Power Adapters',
       ' Car Accessories ', ' Accessory Kits',
       ' Batteries & Battery Packs ', ' Batteries & Battery Packs',
       ' Cables', ' Signal Boosters', ' Smartwatch Accessories ',
       ' Chargers & Power Adapters ', ' Holsters', ' Screen Protectors',
       ' SIM Cards', ' Car Accessories', ' Armbands', ' Flip Cases',
       ' Phone Charms', ' Sleeves', ' Virtual Reality Headsets',
       ' Replacement Parts', ' Mounts & Stands ', ' Mobile Hotspots',
       ' SIM Card Tools & Accessories', ' Cell Phone Minutes',
       ' Wireless Accessories', ' Selfie Sticks & Tripods',
       ' MP3 & MP4 Player Accessories ', ' Single Ear Bluetooth Headsets',
       ' Lens Attachments', ' Dry Bags', ' Mounts & Stands',
       ' Smartwatch Accessories', ' GPS System Accessories',
       ' Mobile Flashes & Selfie Lights', ' Solar & Wind Power ',
       ' Mobile & Wearable Tech', ' Leathercraft', ' Cord Management',
       ' Jump Starters, Bat

In [5]:
df['sub_cat_3'].unique()

array([nan, ' Car Chargers', ' Car Kits', ' Battery Charger Cases',
       ' Car Cradles & Mounts', ' Car Speakerphones',
       ' Smartwatch Screen Protectors', ' Charging Stations',
       ' Wall Chargers', ' Replacement Batteries',
       ' Portable Power Banks', ' Stands', ' Solar Chargers',
       ' Batteries & Battery Packs', ' Car Pads & Mats',
       ' Smartwatch Bands', ' Smartwatch Cables & Chargers',
       ' Smartwatch Cases', ' Solar Battery Chargers & Charging Kits',
       ' Smartwatch Stickers', ' Smartwatch Cases with Band'],
      dtype=object)

In [6]:
df['sub_cat_1'].unique()

array([' Cases, Holsters & Clips', ' Cases, Holsters & Clips ',
       ' Cell Phone Accessories ', ' Cell Phone Accessories',
       ' Carrier Cell Phones', ' Unlocked Cell Phones',
       ' SIM Cards & Prepaid Minutes ', ' Mobile Broadband ', ' Gadgets ',
       ' Office Electronics', ' Outlet', ' Portable Audio & Video ',
       ' Fan Shop', ' SIM Cards & Prepaid Minutes', ' Mobile Broadband',
       ' Toys', ' Portable Audio & Video', ' Accessories & Supplies ',
       ' Generators & Portable Power ', ' GPS & Navigation',
       ' Craft Supplies ', ' Electrical ', ' Tools & Equipment '],
      dtype=object)

In [7]:
cx = [i for i in range(1500,1600)]

In [8]:
#df[df['sub_cat_1'] == ' Carrier Cell Phones'].iloc[cx]

In [9]:
df[df['sub_cat_1'] == ' Tools & Equipment '].head(200)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_rank
876081,5,False,"11 3, 2015",A3JWIN5KVRFVFM,B01B3ISBFS,NaN,technis,I purchased both of the Energen Power Jumper P5 and the Energen Power Jumper P6. Please see the appropriate section for a review of the individual products. Review details line up by paragraph in ...,Review on both P5(10000mAh) and P6(12000mAh | can also charge laptops). Flashlight is great for projects. Jumped V6 in seconds.,1446508800,2,"['https://images-na.ssl-images-amazon.com/images/I/91XAE9Su+oL._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/81DYBydab3L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images...","['<b>Energen Young, Enthusiastic, and Dedicated </b> <br> We create innovative and high quality mobile power accessories. <br> <br> <b>Features</b> <br> Vehicle jump starter. Jump-start engines ...","['Powerful jump starter shrunk into a compact and lightweight portable package. Jump start automobiles, and motorcycles, including many watercrafts', 'Portable battery charger for USB-powered devi...","['>#289,161 in Cell Phones & Accessories (See top 100)', '>#3,131 in Automotive > Tools & Equipment > Jump Starters, Battery Chargers & Portable Power', '>#3,181 in Cell Phones & Accessories > Cel...",Cell Phones & Accessories,"['https://images-na.ssl-images-amazon.com/images/I/41LGVGCBnML._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41BG9xu0oQL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/imag...","['https://images-na.ssl-images-amazon.com/images/I/41LGVGCBnML.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41BG9xu0oQL.jpg', 'https://images-na.ssl-images-amazon.com/images/I/418u7hUI1...",55,513,260,289161,Tools & Equipment,"Jump Starters, Battery Chargers & Portable Power",NaN,3131
876082,5,False,"10 23, 2015",A2RH4U6BZ1WTSN,B01B3ISBFS,NaN,Amy F.,"I am pleased as punch with this Energen Power Jumper! Earlier this week, I hopped into my SUV and it wouldn't start. When my husband came home from work, he helped me jump start it and I took it...",Works great for a Chevy Traverse SUV!,1445558400,48,"['https://images-na.ssl-images-amazon.com/images/I/712FRHQci4L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images/I/71QVYZq657L._SY88.jpg', 'https://images-na.ssl-images-amazon.com/images...","['<b>Energen Young, Enthusiastic, and Dedicated </b> <br> We create innovative and high quality mobile power accessories. <br> <br> <b>Features</b> <br> Vehicle jump starter. Jump-start engines ...","['Powerful jump starter shrunk into a compact and lightweight portable package. Jump start automobiles, and motorcycles, including many watercrafts', 'Portable battery charger for USB-powered devi...","['>#289,161 in Cell Phones & Accessories (See top 100)', '>#3,131 in Automotive > Tools & Equipment > Jump Starters, Battery Chargers & Portable Power', '>#3,181 in Cell Phones & Accessories > Cel...",Cell Phones & Accessories,"['https://images-na.ssl-images-amazon.com/images/I/41LGVGCBnML._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41BG9xu0oQL._SS40_.jpg', 'https://images-na.ssl-images-amazon.com/imag...","['https://images-na.ssl-images-amazon.com/images/I/41LGVGCBnML.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41BG9xu0oQL.jpg', 'https://images-na.ssl-images-amazon.com/images/I/418u7hUI1...",20,149,121,289161,Tools & Equipment,"Jump Starters, Battery Chargers & Portable Power",NaN,3131
876083,5,True,"04 29, 2017",A7BYYSCQHH5F1,B01B3ISBFS,{'Size:': ' 10000mah'},Katmai,Great device. My daughter loves it. She uses it to charge her Note 4 and has started several of her friends cars with it.,Perfect E starter for daughters car,1493424000,0,NaN,"['<b>Energen Young, Enthusiastic, and Dedicated </b> <br> We create innovative and high qua

In [10]:
len(df[df['sub_cat_1'] == ' Tools & Equipment '].index)

17

**Subcat1** <br>

' Cases, Holsters & Clips' - Cases/Cover, battery <br>
' Cases, Holsters & Clips ' - Cases, armband <br>
' Cell Phone Accessories ' - Charger, screen protector <br>
' Cell Phone Accessories' - Cases, receptacle (electric socket), cables, windscreen for ear pieces, handsfree headset, charger, signal strength booster, clips <br>
**' Carrier Cell Phones' - Phones (11662 rows) (subcat2 and subcat3 are all NaN)** <br>
**' Unlocked Cell Phones' - Phones (41968 rows) (subcat2 and subcat3 are all NaN)** <br>
' SIM Cards & Prepaid Minutes ' - Sim Cards <br>
' Mobile Broadband ' - Modem, Hotspot
' Gadgets ' - Holder for phones, Cases, Stone Smart Button <br>
' Office Electronics' - Clip, Screen Protector <br>
**' Outlet' - Wall Charger, Phone, Watch Band (4283 rows) (check more, eg. 1500,1600) (subcat2 and subcat3 are all NaN)** <br> 
' Portable Audio & Video ' - Charger <br>
' Fan Shop' - Case <br>
' SIM Cards & Prepaid Minutes' - Sim Card Tray Ejector Pin, Dog Camera <br>
' Mobile Broadband' - Wireless Hub <br>
' Toys' - Sponge Ball kinda thing <br>
' Portable Audio & Video' - Charger <br>
' Accessories & Supplies ' - Charger <br>
' Generators & Portable Power ' - Charger <br>
' GPS & Navigation' - Cases <br>
' Craft Supplies ' - Cases <br>
' Electrical ' - Clip <br>
' Tools & Equipment ' - Jump Starter <br>

In [11]:
df[df['sub_cat_1'] == ' Carrier Cell Phones']['sub_cat_3'].unique()

array([nan], dtype=object)

In [12]:
for i in df['sub_cat_1'].unique():
    print(i, len(df[df['sub_cat_1'] == i].index))

 Cases, Holsters & Clips 144969
 Cases, Holsters & Clips  282137
 Cell Phone Accessories  418285
 Cell Phone Accessories 35526
 Carrier Cell Phones 11662
 Unlocked Cell Phones 41968
 SIM Cards & Prepaid Minutes  7117
 Mobile Broadband  320
 Gadgets  5026
 Office Electronics 66
 Outlet 4283
 Portable Audio & Video  10721
 Fan Shop 143
 SIM Cards & Prepaid Minutes 56
 Mobile Broadband 20
 Toys 24
 Portable Audio & Video 6
 Accessories & Supplies  22
 Generators & Portable Power  14
 GPS & Navigation 12
 Craft Supplies  488
 Electrical  7
 Tools & Equipment  17


In [29]:
df_phone = df[(df['sub_cat_1'] == ' Carrier Cell Phones') | (df['sub_cat_1'] == ' Unlocked Cell Phones')]

In [30]:
df_phone.reset_index(drop=True, inplace=True)

In [31]:
df_phone

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no,sub_cat_1,sub_cat_2,sub_cat_3,sub_cat_rank
0,1,False,"09 21, 2004",A5JLAU2ARJ0BO,B0000E3GWH,NaN,Old-and-Wise,"This phone is ugly and heavy and has a terrible user interface even for techies like myself. It also drops calls like crazy here in Manhattan. The Moto V600 is a much better phone, takes better ...",Bad reception,1095724800,2,NaN,"[""Compatible with T-Mobile cellular service, Samsung's dual-frequency GSM/GPRS SGH-E715 phone includes a Web browser, digital camera, some organizing software, and a host of other useful features....",[],"['>#3,059,436 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#4,085 in Cell Phones & Accessories > Carrier Cell Phones']",Cell Phones & Accessories,"['https://images-na.ssl-images-amazon.com/images/I/31HAE14Z11L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41HV0MFWG3L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://...","['https://images-na.ssl-images-amazon.com/images/I/31HAE14Z11L.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41HV0MFWG3L.jpg', 'https://images-na.ssl-images-amazon.com/images/I/416VGT1ND...",5,40,31,3059436,Carrier Cell Phones,NaN,NaN,4085
1,4,False,"07 24, 2004",A1ACM1CBGORBN1,B0000E3GWH,NaN,Sherry Berry,"I had the Samsung V205 and then I decided to try the E715 out. FYI to people with battery problems, with these kind of batteries, you must let it drain completely and then charge it.. kind of like...",Pretty good phone.. an improvement from my v205,1090627200,15,NaN,"[""Compatible with T-Mobile cellular service, Samsung's dual-frequency GSM/GPRS SGH-E715 phone includes a Web browser, digital camera, some organizing software, and a host of other useful features....",[],"['>#3,059,436 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#4,085 in Cell Phones & Accessories > Carrier Cell Phones']",Cell Phones & Accessories,"['https://images-na.ssl-images-amazon.com/images/I/31HAE14Z11L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41HV0MFWG3L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://...","['https://images-na.ssl-images-amazon.com/images/I/31HAE14Z11L.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41HV0MFWG3L.jpg', 'https://images-na.ssl-images-amazon.com/images/I/416VGT1ND...",18,184,134,3059436,Carrier Cell Phones,NaN,NaN,4085
2,3,False,"07 6, 2004",A2V48Q03FZUOSD,B0000E3GWH,NaN,Benjamin David,"This is the first Samsung phone I have had and it does not quite measure-up to the V300. I am a TMobile customer and when I finally became too fed up with the V300's terrible camera, TMobile sent...",Not as user friendly as the Motorola V300-,1089072000,11,NaN,"[""Compatible with T-Mobile cellular service, Samsung's dual-frequency GSM/GPRS SGH-E715 phone includes a Web browser, digital camera, some organizing software, and a host of other useful features....",[],"['>#3,059,436 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#4,085 in Cell Phones & Accessories > Carrier Cell Phones']",Cell Phones & Accessories,"['https://images-na.ssl-images-amazon.com/images/I/31HAE14Z11L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41HV0MFWG3L._SX38_SY50_CR,0,0,38,50_.jpg', 'https://...","['https://images-na.ssl-images-amazon.com/images/I/31HAE14Z11L.jpg', 'https://images-na.ssl-images-amazon.com/images/I/41HV0MFWG3L.jpg', 'https://images-na.ssl-images-amazon.com/images/I/416VGT1ND...",9,121,90,3059436,Carrier Cell Phones,NaN,NaN,4085
3,5,True,"04 12, 2004",A1V3TRGWOMA8LC,B0000E3GWH,NaN,James Beswick,"..and I'm from Europe, where the phones and networks are much better than in the States. I have to say that I think Samsung currently have the best software (and I used to be a huge Nokia fan). 

In [33]:
df_phone.to_csv(r'C:\Users\surya\Desktop\Jupyter Python Programs\Amazon Review NLP\Cell_Phone_data_csv.csv', index=False)

In [35]:
df_phone.to_excel(r'C:\Users\surya\Desktop\Jupyter Python Programs\Amazon Review NLP\Cell_Phone_data_excel.xlsx', index=False)